# ANOVA  - Lab

## Introduction

In this lab, you'll get some brief practice generating an ANOVA table (AOV) and interpreting its output. You'll then also perform some investigations to compare the method to the t-tests you previously employed to conduct hypothesis testing.

## Objectives

You will be able to:
* Use ANOVA for testing multiple pairwise comparisons
* Understand and explain the methodology behind ANOVA tests

## Loading the Data

Start by loading in the data stored in the file **ToothGrowth.csv**.

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
import scipy.stats as stats

In [2]:
df = pd.read_csv('ToothGrowth.csv')

In [3]:
print(df.info())
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 3 columns):
len     60 non-null float64
supp    60 non-null object
dose    60 non-null float64
dtypes: float64(2), object(1)
memory usage: 1.5+ KB
None


,len,supp,dose
0,4.2,VC,0.5
1,11.5,VC,0.5
2,7.3,VC,0.5
3,5.8,VC,0.5
4,6.4,VC,0.5
5,10.0,VC,0.5
6,11.2,VC,0.5
7,11.2,VC,0.5
8,5.2,VC,0.5
9,7.0,VC,0.5


## Generating the ANOVA Table

Now generate an ANOVA table in order to analyze the influence of the medication and dosage 

In [4]:
formula = 'len ~ C(supp) + C(dose)'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

               sum_sq    df          F        PR(>F)
C(supp)    205.350000   1.0  14.016638  4.292793e-04
C(dose)   2426.434333   2.0  82.810935  1.871163e-17
Residual   820.425000  56.0        NaN           NaN


## Reading the Table

Make a brief comment regarding the statistics regarding the effect of supplement and dosage on tooth length.

#### Your comment here
dose seems to be the most influential, followed by the supplement variable.

## Comparing to T-Tests

Now that you've gotten a brief chance to interact with ANOVA, its interesting to compare the results to those from the t-tests you were just working with. With that, start by breaking the data into two samples: those given the OJ supplement, and those given the VC supplement. Afterwards, you'll conduct a t-test to compare the tooth length of these two different samples.

In [5]:
df_oj = df.loc[df['supp'] == 'OJ']
df_vc = df.loc[df['supp'] == 'VC']
df_oj

,len,supp,dose
30,15.2,OJ,0.5
31,21.5,OJ,0.5
32,17.6,OJ,0.5
33,9.7,OJ,0.5
34,14.5,OJ,0.5
35,10.0,OJ,0.5
36,8.2,OJ,0.5
37,9.4,OJ,0.5
38,16.5,OJ,0.5
39,9.7,OJ,0.5


Now compare a t-test between these two groups and print the associated two-sided p-value.

In [6]:
# calculate the 2-sided p-value for a t-test comparing the two supplement groups.
df_oj_percent = df_oj.sample(frac=0.7)

In [7]:
def welch_t(a, b):
    """ Calculate Welch's t statistic for two samples. """
    numerator = a.mean() - b.mean()
    denominator = np.sqrt(a.var(ddof=1)/len(a) + b.var(ddof=1)/b.size)
    return abs(numerator/denominator)

def welch_df(a, b):
    """ Calculate the effective degrees of freedom for two samples. """
    num = (a.var(ddof=1)/a.size + b.var(ddof=1)/b.size)**2
    denom = ((a.var(ddof=1)**2) / (((a.size)**2) * (a.size-1))) + ((b.var(ddof=1)**2) / (((b.size)**2) * (b.size-1)))
    return num/denom

def p_value(a, b, two_sided=False):
    t = welch_t(a,b)
    df = welch_df(a,b)
    p = 1 - stats.t.cdf(t, df)
    if two_sided:
        return 2*p
    else:
        return p

In [8]:
p_value(df_oj.len,df_vc.len,two_sided=True)

0.06063450788093383

## A 2-Category ANOVA F-Test is Equivalent to a 2-Tailed t-Test!

Now, recalculate an ANOVA F-test with only the supplement variable. An ANOVA F-test between two categories is the same as performing a 2-tailed t-Test! So, the p-value in the table should be identical to your calculation above.

> Note: there may be a small fractional difference (>0.001) between the two values due to a rounding error between implementations. 

In [9]:
#Conduct an ANOVA F-test of the oj and vc supplement groups.
#Compare the p-value to that of the t-test above. 
#They should match (there may be a tiny fractional difference due to rounding errors in varying implementations)

formula = 'len ~ C(supp)'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

               sum_sq    df         F    PR(>F)
C(supp)    205.350000   1.0  3.668253  0.060393
Residual  3246.859333  58.0       NaN       NaN


## Generating Multiple T-Tests

While the 2-category ANOVA test is identical to a 2-tailed t-Test, performing multiple t-tests leads to the multiple comparisons problem. To investigate this, look at the various sample groups you could create from the 2 features: 

In [10]:
for group in df.groupby(['supp', 'dose'])['len']:
    group_name = group[0]
    data = group[1]
    print(group_name)

('OJ', 0.5)
('OJ', 1.0)
('OJ', 2.0)
('VC', 0.5)
('VC', 1.0)
('VC', 2.0)


While bad practice, examine the effects of calculating multiple t-tests with the various combinations of these. To do this, generate all combinations of the above groups. For each pairwise combination, calculate the p-value of a 2 sided t-test. Print the group combinations and their associated p-value for the two-sided t-test.

In [11]:
from itertools import combinations


# fs.p_value_welch_ttest(oj_lengths, vc_lengths, two_sided=True)


groups = [group[0] for group in df.groupby(['supp', 'dose'])['len']]
combos = combinations(groups, 2)
for combo in combos:
    supp1 = combo[0][0]
    dose1 = combo[0][1]
    supp2 = combo[1][0]
    dose2 = combo[1][1]
    sample1 = df[(df.supp == supp1) & (df.dose == dose1)]
    sample2 = df[(df.supp == supp2) & (df.dose == dose2)]
    p = p_value(sample1, sample2, two_sided=True)
    print(combo, p[0])

(('OJ', 0.5), ('OJ', 1.0)) 1.4866209618880077e-07
(('OJ', 0.5), ('OJ', 2.0)) 3.1072699968603956e-11
(('OJ', 0.5), ('VC', 0.5)) 0.0009894705755875144
(('OJ', 0.5), ('VC', 1.0)) 0.021188235155023882
(('OJ', 0.5), ('VC', 2.0)) 1.5016921040000852e-10
(('OJ', 1.0), ('OJ', 2.0)) 0.014554700356502526
(('OJ', 1.0), ('VC', 0.5)) 3.1086244689504383e-15
(('OJ', 1.0), ('VC', 1.0)) 4.203335666597141e-05
(('OJ', 1.0), ('VC', 2.0)) 0.02710575576515617
(('OJ', 2.0), ('VC', 0.5)) 0.0
(('OJ', 2.0), ('VC', 1.0)) 9.481304630298837e-14


C:\Users\Matthew Parker\.conda\envs\learn-env\lib\site-packages\scipy\stats\_distn_infrastructure.py:872: RuntimeWarning: invalid value encountered in greater
  cond = logical_and(cond, (asarray(arg) > 0))
C:\Users\Matthew Parker\.conda\envs\learn-env\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\Matthew Parker\.conda\envs\learn-env\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\Matthew Parker\.conda\envs\learn-env\lib\site-packages\scipy\stats\_distn_infrastructure.py:1738: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0


(('OJ', 2.0), ('VC', 2.0)) 0.9477213632803463
(('VC', 0.5), ('VC', 1.0)) 1.7352785874891197e-12
(('VC', 0.5), ('VC', 2.0)) 0.0
(('VC', 1.0), ('VC', 2.0)) 5.418798743050957e-10


In [16]:
#Reuse your t-test code above to calculate the p-value for a 2-sided t-test
#for all combinations of the supplement-dose groups listed above. 
#(Since there isn't a control group, compare each group to every other group.)

for drug1 in ['OJ','VC']:
    for drug2 in ['OJ', 'VC']:
        for n in [0.5, 1.0, 2.0]:
            for m in [0.5, 1.0, 2.0]:
                if drug1 == drug2 and n == m:
                    continue
                else:
                    sample1 = df.loc[(df['dose'] == n) & (df['supp'] == drug1)].len
                    sample2 = df.loc[(df['dose'] == m) & (df['supp'] == drug2)].len
                    p = p_value(sample1, sample2, two_sided=True)
                    print(f'{drug1} dosage: {n}\n{drug2} dosage: {m}\np-value:{p}\n')

OJ dosage: 0.5
OJ dosage: 1.0
p-value:8.784919055160323e-05

OJ dosage: 0.5
OJ dosage: 2.0
p-value:1.3237838778490385e-06

OJ dosage: 1.0
OJ dosage: 0.5
p-value:8.784919055160323e-05

OJ dosage: 1.0
OJ dosage: 2.0
p-value:0.03919514204624397

OJ dosage: 2.0
OJ dosage: 0.5
p-value:1.3237838778490385e-06

OJ dosage: 2.0
OJ dosage: 1.0
p-value:0.03919514204624397

OJ dosage: 0.5
VC dosage: 0.5
p-value:0.00635860676409683

OJ dosage: 0.5
VC dosage: 1.0
p-value:0.04601033257637566

OJ dosage: 0.5
VC dosage: 2.0
p-value:7.196253523966689e-06

OJ dosage: 1.0
VC dosage: 0.5
p-value:3.655206715080794e-08

OJ dosage: 1.0
VC dosage: 1.0
p-value:0.0010383758723000458

OJ dosage: 1.0
VC dosage: 2.0
p-value:0.09652612338267019

OJ dosage: 2.0
VC dosage: 0.5
p-value:1.362154833373097e-11

OJ dosage: 2.0
VC dosage: 1.0
p-value:2.3610742005963914e-07

OJ dosage: 2.0
VC dosage: 2.0
p-value:0.9638515887233756

VC dosage: 0.5
OJ dosage: 0.5
p-value:0.00635860676409683

VC dosage: 0.5
OJ dosage: 1.0
p-valu

## Summary

In this lesson, you examined the ANOVA technique to generalize A/B testing methods to multiple groups and factors.